In [15]:
import os
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse as sp
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from sklearn.preprocessing import MinMaxScaler


In [2]:
HOME = os.getcwd()
print(HOME)
ANIME_DATA = os.path.join(HOME, r'Preprocessing\Preprocessed Data\raw_anime_processed.csv')
print(ANIME_DATA)

d:\Data Science Introduction\Final_Project\models\anime_models
d:\Data Science Introduction\Final_Project\models\anime_models\Preprocessing\Preprocessed Data\raw_anime_processed.csv


In [3]:
# df = pd.read_csv(ANIME_DATA)
df = pd.read_csv(r'D:\Data Science Introduction\Final_Project\Preprocessing\Preprocessed Data\raw_anime_processed.csv')
df.head()

,Title,Score,Vote,Ranked,Popularity,Members,Favorite,Types,Volumes,Chapters,...,Demographic,Serialization,Author,Total Review,Type Review,Realeased date,Completed date,Recommended,Mixed Feelings,Not Recommended
0,Berserk,9.47,363720,1,1,725079,130489,Manga,3,15,...,Seinen,Young Animal,"[""'Miura,Kentarou'"", ""'Studio Gaga'""]",289,"[252, 17, 20]",1989-08-25,Updating,252,17,20
1,JoJo no Kimyou na Bouken Part 7: Steel Ball Ru...,9.31,172219,2,23,280428,46269,Manga,24,96,...,Seinen,Ultra Jump,"[""'Araki,Hirohiko'""]",131,"[123, 7, 1]",2004-01-19,2011-04-19,123,7,1
2,Vagabond,9.26,154583,3,13,406082,44258,Manga,37,327,...,Seinen,Morning,"[""'Inoue,Takehiko'"", ""'Yoshikawa,Eiji'""]",104,"[93, 9, 2]",1998-09-03,2015-05-21,93,9,2
3,One Piece,9.22,392811,4,4,642620,119974,Manga,3,15,...,Shounen,Shounen Jump (Weekly),"[""'Oda,Eiichiro'""]",231,"[190, 21, 20]",1997-07-22,Updating,190,21,20
4,Monster,9.16,104327,5,29,258581,22008,Manga,18,162,...,Seinen,Big Comic Original,"[""'Urasawa,Naoki'""]",86,"[69, 11, 6]",1994-12-05,2001-12-20,69,11,6


Attributes  dataset

In [4]:
df.keys()

Index(['Title', 'Score', 'Vote', 'Ranked', 'Popularity', 'Members', 'Favorite',
       'Types', 'Volumes', 'Chapters', 'Status', 'Published', 'Genres',
       'Themes', 'Demographic', 'Serialization', 'Author', 'Total Review',
       'Type Review', 'Realeased date', 'Completed date', 'Recommended',
       'Mixed Feelings', 'Not Recommended'],
      dtype='object')

Filling empty values

In [5]:
df['Genres'] = df['Genres'].fillna(str(['''''']))
df['Themes'] = df['Themes'].fillna(str(['''''']))

Normalize string attributes

In [ ]:
# Chuyển các thuộc tính dạng list thành chuỗi văn bản
df['Genres_str'] = df['Genres'].apply(lambda x: " ".join([genre.strip("'") for genre in x.split(',')]) if x and isinstance(x, str) else "")
df['Themes_str'] = df['Themes'].apply(lambda x: " ".join([theme.strip("'") for theme in x.split(',')]) if x and isinstance(x, str) else "")
df['Author_str'] = df['Author'].apply(lambda x: " ".join([author.strip("'") for author in x.split(',')]) if x and isinstance(x, str) else "")

# Áp dụng TF-IDF cho các thuộc tính dạng chuỗi
tfidf = TfidfVectorizer(stop_words='english')

tfidf_Title_matrix = tfidf.fit_transform(df['Title'])
tfidf_Types_matrix = tfidf.fit_transform(df['Types'])
tfidf_Genres_matrix = tfidf.fit_transform(df['Genres_str'])
tfidf_Themes_matrix = tfidf.fit_transform(df['Themes_str'])
tfidf_Author_matrix = tfidf.fit_transform(df['Author_str'])

Normalize number attributes

In [ ]:

# Chuẩn hóa các thuộc tính dạng số
scaler = MinMaxScaler()
df[['Score', 'Vote', 'Ranked', 'Popularity', 'Recommended']] = scaler.fit_transform(df[['Score', 'Vote', 'Ranked', 'Popularity', 'Recommended']])


In [ ]:
X_tfidf = sp.hstack([
    tfidf_Title_matrix,       # Giữ dạng sparse
    tfidf_Types_matrix,
    tfidf_Genres_matrix,
    tfidf_Themes_matrix,
    tfidf_Author_matrix,
    sp.csr_matrix(df[['Score']].values),  # Chuyển các cột số sang sparse
    sp.csr_matrix(df[['Vote']].values),
    sp.csr_matrix(df[['Ranked']].values),
    sp.csr_matrix(df[['Popularity']].values),
    sp.csr_matrix(df[['Recommended']].values)
])


In [ ]:
# Tính ma trận độ tương đồng cosine
cosine_sim = cosine_similarity(X_tfidf, X_tfidf)


MemoryError: Unable to allocate 2.98 GiB for an array with shape (400000000,) and data type float64

Attributes selection using PCA

In [10]:
def recommend_anime(title, cosine_sim, df, top_n=5):
    # Step 1: Find the index (anime_id) of the anime matching the title
    idx = df[df['Title'] == title].index

    if idx.empty:
        print(f"No anime found with title: {title}")
        return []
    
    idx = idx[0]  # Get the first match (assuming titles are unique)

    # Step 2: Get the similarity scores for this anime
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Step 3: Sort by similarity scores in descending order
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Step 4: Get the top_n most similar anime (excluding itself)
    sim_scores = sim_scores[1:top_n+1]  # Skip the first, as it’s the same anime

    # Step 5: Retrieve the anime titles for recommendations
    anime_indices = [i[0] for i in sim_scores]
    recommended = df.iloc[anime_indices]

    return recommended[['Title', 'Score']]  # Return relevant columns


In [12]:
title_to_search = "Yotsuba to! (Yotsuba&!)"
recommendations = recommend_anime(title_to_search, cosine_sim, df, top_n=5)

if not recommendations.empty:
    print("Recommended anime:")
    print(recommendations)
else:
    print("No recommendations found.")


Recommended anime:
                                             Title     Score
26    3-gatsu no Lion (March Comes in Like a Lion)  0.913352
55      Shoujo Shuumatsu Ryokou (Girls' Last Tour)  0.892045
1981                   Akachan to Boku (Baby & Me)  0.741477
247                                 Azumanga Daioh  0.842330
3064                                  Papa Told Me  0.717330
